### 웹 크롤링

#### requests

백그라운드에서 웹 크롤링을 진행.  

    requests.get("url")
    
입력한 URL을 호출함.

**<HTTP 응답 번호>**
- 200은 성공(205: ???)  
- 300번대는 리다이렉션 메시지  
- 400번대는 오류 메시지(403: 권한 없음, 404: 존재하지 않는 페이지)

웹페이지 개발자 도구 이용  
보통은 개발자 도구 페이지의 개체를 마우스 우클릭하면 나오는 XPath를 사용하여 코드 작성.

#### bs4

con.content

In [5]:
import urllib as ulb
import requests as rqs
from bs4 import BeautifulSoup as bs

url = "https://ko.wikipedia.org/wiki/조선_세종"

a = rqs.get("http://www.naver.com")
b = bs(a.content, "lxml")

lst_kw = b.find_all(name = "a", class_ = "ah_k", attr = {"data-clk" : "lve.keyword"})
b
# lst_rk = b.fine_all(name = "span", class_ = "ah_r")

# for line in :
    # print(line.get_text())

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="origin" name="Referrer"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1100" name="viewport"/>
<meta content="NAVER" name="apple-mobile-web-app-title"/>
<meta content="index,nofollow" name="robots"/>
<meta content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요" name="description"/>
<meta content="네이버" property="og:title"/>
<meta content="https://www.naver.com/" property="og:url"/>
<meta content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png" property="og:image"/>
<meta content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요" property="og:description"/>
<meta content="summary" name="twitter:card"/>
<meta content="" name="twitter:title"/>
<meta content="https://www.naver.com/" name="twitter:url"/>
<meta content="https://s.pstatic.net/static/www/mobil

파이썬으로 매일 오전 8시, 오후 8시 실시간 검색어를 7일간 크롤링하기.  
검색어의 순위 정보도 크롤링돼야 한다.

In [12]:
import urllib as ulb
import requests as rqs
from bs4 import BeautifulSoup as soup

In [13]:
date = ["2019-05-16", "2019-05-17", "2019-05-18", "2019-05-19", "2019-05-20", "2019-05-21", "2019-05-22", "2019-05-23"]
time = ["08%3A00%3A00", "20%3A00%3A00"]

url = "https://datalab.naver.com/keyword/realtimeList.naver?datetime=%sT%s&where=main" %(date[0], time[1])
print(url)

rq_header = {
    "User-Agent" : ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.134 Safari/537.36 Vivaldi/2.5.1525.40")
}

page = rqs.get(url, headers = rq_header)
page.encoding = "utf-8"
print(page.url)
# print(page.history)

source_page = soup(page.content, "lxml")

https://datalab.naver.com/keyword/realtimeList.naver?datetime=2019-05-16T20%3A00%3A00&where=main
https://datalab.naver.com/keyword/realtimeList.naver?datetime=2019-05-16T20%3A00%3A00&where=main


In [11]:
def export_to_sql(d, t, k, r, c, s):
    dict_age = {"all" : "전체", "10s" : "10대", "20s" : "20대", "30s" : "30대", "40s" : "40대", "50s" : "50대 이상"}
    
    dict_time = {"08%3A00%3A00": "08:00", "20%3A00%3A00": "20:00"}
    
    sql = """INSERT INTO Search_Keywords(Date, Time, Keyword, Rank, Category, Source)
VALUES ('%s', '%s', '%s', '%s', '%s', '%s');""" %(d, dict_time[t], k, r, dict_age[c], s)
        
    return sql
    
    
def crawler(dates, times):
    rq_header = {
    "User-Agent" : ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.134 Safari/537.36 Vivaldi/2.5.1525.40")
    }
    
    ages = ["all", "10s", "20s", "30s", "40s", "50s"]
    
    for d in dates:  # 일자별로 키워드 수집
        for t in times:  # 시간별로 키워드 수집
            url = "https://datalab.naver.com/keyword/realtimeList.naver?datetime=%sT%s&where=main" %(d, t)
            
            page = rqs.get(url, headers = rq_header)
            page.encoding = "utf-8"
            
            source_page = soup(page.content, "lxml")
            
            for i in range(len(ages)):  # 나이별로 키워드 수집
                keywords = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='%s'] span" %ages[i])
                
                kwds_txt = []
                for j in range(len(keywords)):  # 키워드(텍스트)만 추출하여 txt에 추가
                    kwds_txt.append(keywords[j].text)
                
                for j in range(len(kwds_txt)):
                    sql = export_to_sql(d, t, kwds_txt[j], j + 1, ages[i], "Naver")  # Date, Time, Keyword, Rank, Category, Source
                    cursor.execute(sql)
    
    print("end")

    cnx.commit()
    print("커밋됨.")

    cnx.close()
    print("종료됨.")
    
    return 0

In [14]:
dates = ["2019-05-16", "2019-05-17", "2019-05-18", "2019-05-19", "2019-05-20", "2019-05-21", "2019-05-22", "2019-05-23"]

times = ["08%3A00%3A00", "20%3A00%3A00"]

crawler(["2019-05-23"], ["08%3A00%3A00"])

end
커밋됨.
종료됨.


0

In [16]:
# keywords = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank > .rank_inner *")
# rank = source_page
keywords_all = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='all'] span")

res_all = []
for i in range(len(keywords_all)):
    res_all.append(keywords_all[i].text)

res_all

['김기태',
 '유승현',
 '생생정보통 이피디',
 '이미자',
 '도니도니돈까스',
 '김현아 의원',
 '한센병',
 '나이키 사카이',
 '이재명',
 '박흥식',
 '절대그이',
 '구형',
 '강유미',
 '김성수',
 '누룽지통닭',
 '엠카운트다운',
 '이동휘',
 '드라이카레',
 '레이디스코드',
 '문무일']

In [17]:
keywords_10s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='10s'] span")

res_10s = []
for i in range(len(keywords_10s)):
    res_10s.append(keywords_10s[i].text)

res_10s

['김기태',
 '구형',
 '에이틴',
 '김성수',
 '엠카운트다운',
 '내신등급계산기',
 '타미힐피거 반팔',
 '레이디스코드',
 '체육대회 머리',
 '이동휘',
 '사형 구형',
 '연애혁명',
 '브이앱',
 '강서구 pc방 살인',
 '네이버티비',
 '박흥식',
 'ebsi 고등',
 '2019 인하대 축제',
 '메트로시티 팔찌',
 '승리 기각']

In [18]:
keywords_20s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='20s'] span")

res_20s = []
for i in range(len(keywords_20s)):
    res_20s.append(keywords_20s[i].text)

res_20s

['김기태',
 '이동휘',
 '구형',
 '엠카운트다운',
 '나이키 사카이',
 '메이플 인벤',
 '김성수',
 '생생정보통 이피디',
 '강유미',
 '레이디스코드',
 '도니도니돈까스',
 '절대그이',
 '타미힐피거 반팔',
 '사형 구형',
 '박흥식',
 '승리 기각',
 '강서구 pc방 살인',
 '2019 홍대 축제',
 '유승현',
 '옥택연']

In [7]:
keywords_30s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='30s'] span")

res_30s = []
for i in range(len(keywords_30s)):
    res_30s.append(keywords_30s[i].text)

res_30s

['김기태',
 '도니도니돈까스',
 '생생정보통 이피디',
 '나이키 사카이',
 '유승현',
 '절대그이',
 '강유미',
 '이재명',
 '드라이카레',
 '한센병',
 '누룽지통닭',
 '이상화',
 '박흥식',
 '송도 사고',
 '레이디스코드',
 '장민재',
 '임지현',
 '김성수',
 '이미자',
 '대왕타르트']

In [8]:
keywords_40s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='40s'] span")

res_40s = []
for i in range(len(keywords_40s)):
    res_40s.append(keywords_40s[i].text)

res_40s

['유승현',
 '생생정보 이피디',
 '김기태',
 '한센병',
 '김현아 의원',
 '이재명',
 '도니도니돈까스',
 '이미자',
 '박흥식',
 '누룽지통닭',
 '절대그이',
 '드라이카레',
 '송도 사고',
 '나이키 사카이',
 '심재철',
 '참나무장작구이 누룽지통닭',
 '대왕타르트',
 '아이엠넘버포',
 '생방송투데이',
 '마라롱샤']

In [11]:
keywords_50s = source_page.select(".section_lst_area.carousel_area.lst_narrow .keyword_rank div[data-age='50s'] span")

res_50s = []
for i in range(len(keywords_50s)):
    res_50s.append(keywords_50s[i].text)

res_50s

['이미자',
 '유승현',
 '김현아 의원',
 '생생정보 이피디',
 '한센병',
 '문무일',
 '이재명 재판',
 '심재철',
 '김기태',
 '박흥식',
 '가수이미자나이',
 '누룽지통닭',
 '아미스타드',
 '생방송투데이',
 '참나무장작구이 누룽지통닭',
 '2tv 저녁 생생정보',
 '드라이카레',
 '이지윤 pd',
 '홍정욱',
 '절대그이']

In [10]:
import pymysql as pms

cnx = pms.connect(host = "plen-tutoring.mysql.database.azure.com", 
                  port = 3306, 
                  user = "plentir@plen-tutoring", 
                  password = "yjy990810!", 
                  database = "web_crawling", 
                  charset = "utf8")

cursor = cnx.cursor()

In [6]:
mk_table = """CREATE TABLE Search_Keywords(
ID INT UNSIGNED NOT NULL AUTO_INCREMENT, 
Date VARCHAR(10) NOT NULL, 
Time VARCHAR(8) NOT NULL, 
Keyword VARCHAR(255) NOT NULL, 
Rank INT NOT NULL, 
Category VARCHAR(20) NOT NULL, 
Source VARCHAR(50) NOT NULL, 
PRIMARY KEY(ID)
) DEFAULT CHARSET = utf8 AUTO_INCREMENT = 1;
"""

cursor.execute(mk_table)

0